# Running Constraint Scan

In [1]:
# | default_exp runconstraintscan

This notebook can either be run as a notebook, or using nbdev a python script is produced that can be run in the command line for easier running on e.g. a cluster. It is a notebook in the nbs directory and exported as a .py file in the mmon-gcm library.

In [2]:
# | export

import sys
import os

import cobra
import mmon_gcm.constraintscan  # from x import y syntax doesn't work because of nbdev export format
import pandas as pd
from pandarallel import pandarallel

In [3]:
# | export

test = False

In [4]:
# This cell isn't exported to the .py file, so define here if running in notebook rather than as .py on e.g.a cluster
# This is also where to change whether the light is blue or white if you want to run in the notebook

sys.argv = [
    "script_name",
    "../outputs/constraint_scan/constraint_scan_results_white.csv",
    "../models/4_stage_GC.json",
    "../outputs/constraint_scan/constraints_df.csv",
    "white",
    "6",
    "False"
]

In [5]:
# | export

results_path = sys.argv[1]
model_path = sys.argv[2]
constraints_csv = sys.argv[3]
light_colour = sys.argv[4]
no_cores = int(sys.argv[5])
inc_maintenance = eval(sys.argv[6])

In [6]:
# | export
constraints = pd.read_csv(constraints_csv, index_col=[0], header=[0])
model = cobra.io.load_json_model(model_path)

**If you are running from notebook you'll want to comment out the next cell! It's here so that running nbdev tests doesn't take too long, as they test running the entire notebook. It won't be exported so running as a script will be fine**

In [7]:
#test == True
#constraints = constraints.iloc[:10]  # so tests don't take too long

In [8]:
constraints.head()

,P_abs,T_l,A_l,V_gc_ind,FqFm,R_ch,R_ch_vol,L_air,L_epidermis,Vac_frac,...,N_gcs,n,m,r,s,C_apo,A_closed,A_open,ATPase,Maintenance
0,0.821263,0.000219,1.0,1.985531e-12,0.893439,0.164738,0.211405,0.187493,0.184501,0.895672,...,2.415771e+08,2.491444,0.909122,5.355413e-14,2.296275e-13,0.023939,2.303029,5.442436,8.959799,0.000815
1,0.941372,0.000178,1.0,3.598954e-12,0.898505,0.154986,0.210076,0.348168,0.210304,0.819755,...,7.175430e+08,2.159945,0.881894,6.420317e-14,2.166552e-13,0.025459,3.557535,11.198578,6.451445,0.002891
2,0.977125,0.000173,1.0,2.840697e-12,0.843482,0.169498,0.210506,0.185908,0.143417,0.790013,...,9.187141e+08,2.201635,0.993783,5.726095e-14,1.284962e-13,0.035615,1.607276,4.894487,0.925869,0.000655
3,0.837272,0.000216,1.0,2.785511e-12,0.837985,0.075850,0.212841,0.205723,0.171366,0.774391,...,8.060406e+08,1.886525,0.977332,6.064119e-14,1.612963e-13,0.024588,1.535557,3.282600,7.271118,0.001314
4,0.924047,0.000180,1.0,2.909005e-12,0.810827,0.180420,0.211274,0.332073,0.120961,0.886934,...,5.123453e+08,1.961033,0.928857,5.519750e-14,2.957881e-13,0.035039,3.362713,11.709579,13.615575,0.001114


In [9]:
print(f"Number of constraints = {len(constraints.index)}")

Number of constraints = 965


### Breakdown into batches of constraints

In [10]:
# | export

# Define the batch size
batch_size = 100

df = constraints
# Calculate the number of batches
num_batches = len(df) // batch_size + 1

# Iterate through batches
for batch_number in range(num_batches):
    # Calculate the start and end indices for the current batch
    start_index = batch_number * batch_size
    end_index = (batch_number + 1) * batch_size

    # Slice the DataFrame for the current batch
    current_batch = df.iloc[start_index:end_index]

    if not os.path.exists(f"{results_path[:-4]}_{batch_number+1}.csv"):
        pandarallel.initialize(nb_workers=no_cores, progress_bar=False)
        #scan_solution = constraints.parallel_apply(
        scan_solution = current_batch.parallel_apply(
            mmon_gcm.constraintscan.solve_model_for_constraint_scan,
            args=([model, light_colour, False, inc_maintenance]),
            axis=1,
        )

        solution_df = pd.DataFrame()
        for i, solution in scan_solution.items():
            # this is to catch infeasible solutions
            try:
                solution_df = solution_df.append(
                    solution.to_frame().loc[:, "fluxes"], ignore_index=True
                )
            except:
                solution_df = solution_df.append(pd.Series(dtype=float), ignore_index=True)

        solution_df.to_csv(f"{results_path[:-4]}_{batch_number+1}.csv")

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Solving model 0
Solving model 17
Solving model 34
Solving model 54
Solving model 71
Solving model 88
Solved model 0
Solved model 17
Solving model 1
Solved model 88
Solved model 54
Solved model 34
Solving model 18
Solved model 71
Solving model 89
Solving model 55
Solving model 35
Solving model 72
Solved model 1
Solved model 18
Solved model 89
Solved model 72
Solved model 55
Solved model 35
Solving model 2
Solving model 19
Solving model 90
Solving model 74
Solving model 56
Solving model 36
Solved model 90
Solved model 2
Solved model 74
Solved model 19
Solved model 56
Solving model 91
Solving model 3
Solving model 75
Solving model 20
Solved model 36
Solving model 57
Solving model 37
Solved model 91
Solved model 20
Solved model 57
Solved model 3
Solved model 75
Solving model 92
Solved model 37
Solving model 58
Solving model 21
Solving model 4
Solving

/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Solving model 105
Solving model 123
Solving model 140
Solving model 157
Solving model 174
Solving model 190
Solved model 105
Solved model 123
Solved model 190
Solved model 157
Solved model 174
Solved model 140
Solving model 106
Solving model 124
Solving model 191
Solving model 158
Solving model 175
Solving model 141
Solved model 106
Solved model 191
Solved model 124
Solved model 158
Solved model 175
Solving model 107
Solved model 141
Solving model 125
Solving model 192
Solving model 159
Solving model 176
Solving model 142
Solved model 107
Solved model 159
Solved model 176
Solved model 192
Solved model 142
Solving model 108
Solved model 125
Solving model 160
Solving model 177
Solving model 193
Solving model 143
Solving model 126
Solved model 108
Solved model 160
Solving model 110
Solving model 161
Solved model 177
Solved model 143
Solved model 193

/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

...

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Solving model 830Solving model 848

Solving model 865
Solving model 883
Solving model 901
Solving model 917
Solved model 848
Solved model 830
Solved model 865
Solved model 917
Solved model 901
Solving model 849
Solving model 831
Solving model 866
Solved model 883
Solving model 918
Solving model 902
Solving model 884
Solved model 849
Solved model 866
Solved model 918Solved model 902

Solved model 831
Solved model 884
Solving model 850
Solving model 867
Solving model 903
Solving model 919
Solving model 832
Solving model 885
Solved model 867
Solved model 850Solved model 903

Solved model 885
Solved model 919
Solved model 832
Solving model 868
Solving model 904
Solving model 851
Solving model 920
Solving model 886
Solving model 833
Solved model 904
Solved model 868
Solved model 851
Solving model 905
Solving model 869
Solving model 852
Solved model 88

/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Solving model 933
Solving model 945
Solving model 956
Solving model 967
Solving model 978
Solving model 990
Solved model 933
Solved model 967
Solved model 945
Solved model 978
Solved model 990
Solved model 956
Solving model 934
Solving model 968
Solving model 946
Solving model 979
Solving model 991
Solving model 957
Solved model 957
Solved model 934
Solved model 968
Solved model 979
Solved model 946
Solved model 991
Solving model 958
Solving model 969
Solving model 935
Solving model 980
Solving model 947
Solving model 992
Solved model 980
Solved model 969Solved model 947

Solved model 935
Solved model 958
Solving model 982
Solving model 948
Solving model 970
Solving model 959
Solving model 936
Solved model 992
Solving model 993
Solved model 982
Solved model 948
Solved model 959
Solved model 970
Solving model 983
Solved model 936
Solving model 949

/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_13784/1020900625.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

In [11]:
if os.path.exists(f"{results_path[:-4]}_{num_batches}.csv"):
    directory_path = os.path.dirname(results_path)
    # Get a list of all files in the directory
    all_files = os.listdir(directory_path)
    # Filter the list to include
    csv_files = [file for file in all_files if file.startswith(f"{results_path[:-4]}_".rsplit("/")[-1])]

    # Sort the list using a lambda function and underscore as separator
    csv_files = sorted(csv_files, key=lambda x: int(x.rsplit('_')[-1].split('.')[0]))

    # Initialize an empty list to store DataFrames
    dfs = []

    # Read each CSV file into a DataFrame and append it to the list
    for csv_file in csv_files:
        file_path = os.path.join(directory_path, csv_file)
        df = pd.read_csv(file_path)
        dfs.append(df)

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(dfs, ignore_index=True)

    # Remove 'Unnamed: 0' column
    if 'Unnamed: 0' in combined_df.columns:
        combined_df = combined_df.drop('Unnamed: 0', axis=1)
        
    # Print or use the combined DataFrame as needed
    combined_df.to_csv(results_path)
    
    for csv_file in list(csv_files):
        file_path = os.path.join(directory_path, csv_file)
        if os.path.exists(file_path):
            os.remove(file_path)

In [12]:
combined_df.shape

(965, 7097)